In [1]:
import pickle
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [2]:
chromePath ="C:\seleniumdriver\chromedriver-win64\chromedriver-win64\chromedriver.exe"

In [17]:
# Lấy cookie
driver=webdriver.Chrome(chromePath)
url = 'https://mbasic.facebook.com/'
driver.get(url)
time.sleep(3)

email_box = driver.find_element(by=By.XPATH, value='//*[@id="m_login_email"]')
email_box.send_keys("100054308900987")
password_box = driver.find_element(by=By.XPATH, value='//*[@id="password_input_with_placeholder"]/input')
password_box.send_keys("D19641ziCqoK")
password_box.send_keys(Keys.ENTER)
time.sleep(60)

with open('my_cookie.pkl','wb') as file:
    pickle.dump(driver.get_cookies(), file)

driver.close()

C:\Users\vuong\AppData\Local\Temp\ipykernel_16020\2128556931.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(chromePath)


In [ ]:
/html/body/div/div/div[2]/div/div[1]/div[4]/div/a

/html/body/div/div/div[2]/div/div[1]/div[4]/section/article[7]/footer/div[2]/a[1]

/html/body/div/div/div[2]/div/div[1]/div[1]/div/div/div[1]

/html/body/div/div/div[2]/div/div[1]/div[3]/section/article[1]/footer/div[2]/a[1]
/html/body/div/div/div[2]/div/div[1]/div[4]/div/a

/html/body/div/div/div[2]/div/div[1]/div[4]/section/article[7]/footer/div[2]/a[1]

/html/body/div/div/div[2]/div/div[1]/div[1]/div/div/div[1]

/html/body/div/div/div[2]/div/div[1]/div[4]/div/a

In [36]:
def load_more_posts(driver):
    button = driver.find_element(by=By.XPATH, value='/html/body/div/div/div[2]/div/div[1]/div[4]/div/a')
    button.click()

In [37]:
def get_post_link(driver):
    post_links = []
    while True:
        for i in range(1,8):
            try:
                xpath = f"/html/body/div/div/div[2]/div/div[1]/div[4]/section/article[{i}]/footer/div[2]/a[1]"
                post_links.append(driver.find_element(by=By.XPATH, value=xpath).get_attribute('href'))
            except:
                break
        try:
            time.sleep(5)
            load_more_posts(driver)
        except:
            break
    return post_links

In [38]:
def get_post_content(driver, post_link):
    driver.get(post_link)
    xpath = "/html/body/div/div/div[2]/div/div[1]/div[1]/div/div/div[1]"
    content = driver.find_element(by=By.XPATH, value=xpath).text
    return content

In [39]:
group_links = 'https://mbasic.facebook.com/groups/257768141347267'
driver=webdriver.Chrome(chromePath)
driver.get(group_links)

cookies = pickle.load(open("my_cookie.pkl","rb"))
for cookie in cookies:
    driver.add_cookie(cookie)
driver.get(group_links)
time.sleep(3)

post_links = get_post_link(driver)
driver.close()

C:\Users\vuong\AppData\Local\Temp\ipykernel_16020\972977712.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(chromePath)


WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=120.0.6099.130)
Stacktrace:
	GetHandleVerifier [0x00007FF615EE2142+3514994]
	(No symbol) [0x00007FF615B00CE2]
	(No symbol) [0x00007FF6159A76AA]
	(No symbol) [0x00007FF61598E1E9]
	(No symbol) [0x00007FF61598F7CE]
	(No symbol) [0x00007FF6159A7CC3]
	(No symbol) [0x00007FF615980580]
	(No symbol) [0x00007FF615A22EA1]
	(No symbol) [0x00007FF615A15FB0]
	(No symbol) [0x00007FF6159E4BDC]
	(No symbol) [0x00007FF6159E5C64]
	GetHandleVerifier [0x00007FF615F0E16B+3695259]
	GetHandleVerifier [0x00007FF615F66737+4057191]
	GetHandleVerifier [0x00007FF615F5E4E3+4023827]
	GetHandleVerifier [0x00007FF615C304F9+689705]
	(No symbol) [0x00007FF615B0C048]
	(No symbol) [0x00007FF615B08044]
	(No symbol) [0x00007FF615B081C9]
	(No symbol) [0x00007FF615AF88C4]
	BaseThreadInitThunk [0x00007FFB97AD257D+29]
	RtlUserThreadStart [0x00007FFB9838AA

In [40]:
len(post_links)

3072

In [45]:
driver=webdriver.Chrome(chromePath)
driver.get(group_links)

cookies = pickle.load(open("my_cookie.pkl","rb"))
for cookie in cookies:
    driver.add_cookie(cookie)
driver.get(group_links)
time.sleep(3)

contents = []
for link in post_links:
    try:
        content = get_post_content(driver, link)
        contents.append(content)
        time.sleep(1)
    except:
        contents.append('')

C:\Users\vuong\AppData\Local\Temp\ipykernel_16020\661838019.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(chromePath)


In [46]:
len(contents)

3072

In [47]:
# Lưu dữ liệu vào file
df = pd.DataFrame(zip(contents, post_links), columns=['Contents', 'Links'])
file_name = "mlcoban.csv"
df.to_csv(file_name, sep='\t')

In [48]:
df

,Contents,Links
0,"Công cụ Mocap AI tạo hoạt ảnh khuôn mặt, chuyể...",https://mbasic.facebook.com/groups/machinelear...
1,"Chào mọi người, em đang làm quen với ML cơ bản...",https://mbasic.facebook.com/groups/machinelear...
2,"Merger 2 câu để tạo 1 câu mới.\nTrong NLP, mọi...",https://mbasic.facebook.com/groups/machinelear...
3,"Today, we introduce VinaLLaMA, a Vietnamese Fo...",https://mbasic.facebook.com/groups/machinelear...
4,có bác nào làm genai mà cũng hay dùng transfor...,https://mbasic.facebook.com/groups/machinelear...
...,...,...
3067,[Chia sẻ lời giải cuộc thi Kalapa's CreditScor...,https://mbasic.facebook.com/groups/machinelear...
3068,"Chào mọi người, mình có trò chuyện với bác Jef...",https://mbasic.facebook.com/groups/machinelear...
3069,Mình xin chia sẻ Note cá nhân về ngành Bioinfo...,https://mbasic.facebook.com/groups/machinelear...
3070,"chào các bạn, mình muốn tìm một quyển machine ...",https://mbasic.facebook.com/groups/machinelear...
